### Required Libraries

In [10]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,roc_curve
from functions import decile_analysis
print('********* Required Libraries Imported *********')

********* Required Libraries Imported *********


### Load Dataset

In [11]:
#load model developing data
model_data = pd.read_parquet('Leobrix_Release_BBILAP_Combined_Data.parquet.gzip')
print('********* Data loaded ********* ')
print(model_data.shape)
# Make sure no duplicates in the data
model_data =  model_data.drop_duplicates(subset=['unique_reference_number'])
model_data =  model_data.drop_duplicates(subset=['latest_feature_id'])
print(model_data.shape)

********* Data loaded ********* 
(478328, 2831)
(478328, 2831)


### Drop Null target records

In [12]:
model_data['60_in_15mob'].value_counts(dropna=False)

60_in_15mob
NaN    354961
0.0    109935
1.0     13432
Name: count, dtype: int64

In [13]:
# Drop Null values of Target 
df = model_data.dropna(subset=['60_in_15mob'])
# take one copy for model development data
data = df.copy()
#some bank data have inf very rate but by safe replace 
data.replace([np.inf, -np.inf], np.nan, inplace=True)

In [14]:
df['60_in_15mob'].value_counts(dropna=False)

60_in_15mob
0.0    109935
1.0     13432
Name: count, dtype: int64

In [15]:
df['user_segment'].value_counts()

user_segment
OFFUS    90704
ONUS     32663
Name: count, dtype: int64

### Split the Time interval

In [16]:
model_df1 = data[(data['month_year']>='2021-01') & (data['month_year']<='2023-03')]
oot_df1= data[(data['month_year']>='2023-04') & (data['month_year']<='2023-06')]

In [17]:
model_df1.shape, oot_df1.shape

((101984, 2831), (17343, 2831))

In [18]:
features_list = ['Num_Enq_all_loans_last_6M',
'num_PERSONAL LOAN_enq_last_36M',
'num_CONSUMER LOAN_enq_last_9M',
'num_CREDIT CARD_enq_last_3M',
'Total_outstanding_Balance_CREDIT CARD_opened_last_9M',
'highest_sanction_amount_credit_cards',
'Age_of_oldest_open_account',
 'Utilization_CREDIT CARD_opened_last_36M',
'total_credit_all_open',
'num_PL_LE_1_Lac_opened_in_last_3M',
    
# Reconcilated Features
'percentage_ever_delinquent', 
'months_since_most_recent_delinquency_installment', 
'percentage_utilization_gt_75',
'No of enq in past 90days',
'Num_Enq_non_BANK_last_12M',
'Num_of_installment_loan_opened',
'month_year',
'60_in_15mob']

In [19]:
# Selected Features for model build purpose
model_df = model_df1[features_list]

In [20]:
oot_df = oot_df1[features_list]

In [21]:
model_df.shape, oot_df.shape

((101984, 18), (17343, 18))

In [22]:
model_df['60_in_15mob'].value_counts(dropna=False)

60_in_15mob
0.0    90856
1.0    11128
Name: count, dtype: int64

In [23]:
oot_df['60_in_15mob'].value_counts(dropna=False)

60_in_15mob
0.0    15505
1.0     1838
Name: count, dtype: int64

In [24]:
# Separate features (X) and target (y) for Model and OOT datasets
X_model = model_df.drop(columns=['month_year', '60_in_15mob']) # Adjust 'target' to your actual target column name
y_model = model_df['60_in_15mob']

In [25]:
X_oot = oot_df.drop(columns=['month_year', '60_in_15mob'])
y_oot = oot_df['60_in_15mob']

In [26]:
#Handling Null Values
for i in X_model.columns:
    X_model[i].fillna(-9999,inplace=True)

/tmp/ipykernel_3028063/2233446063.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_model[i].fillna(-9999,inplace=True)


In [27]:
for i in X_oot.columns:
    X_oot[i].fillna(-9999,inplace=True)

/tmp/ipykernel_3028063/1979277635.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_oot[i].fillna(-9999,inplace=True)


In [28]:
X_oot.shape,y_oot.shape

((17343, 16), (17343,))

### Split Model dataset into Train and Test

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X_model, y_model, test_size=0.40,random_state=42)

In [30]:
X_train.shape, X_test.shape

((61190, 16), (40794, 16))

In [31]:
#Split Training data into 80% train and 20% eval
X_train_new, X_eval, y_train_new, y_eval = train_test_split(
    X_train, y_train, test_size=0.2, random_state =42
)
print(X_train_new.shape, X_eval.shape, X_test.shape)

(48952, 16) (12238, 16) (40794, 16)


In [32]:
# 2. **Prepare DMatrix for XGBoost**
dtrain = xgb.DMatrix(X_train_new, label=y_train_new)
deval = xgb.DMatrix(X_eval, label=y_eval)
dtest = xgb.DMatrix(X_test, label=y_test)
doot = xgb.DMatrix(X_oot, label=y_oot)

In [33]:
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'learning_rate': 0.01,
     'max_depth': 2,
    'subsample': 0.6,
    'colsample_bytree': 0.6,

    'lambda': 7,
    'alpha': 4.5,
    'gamma': 4.8,
   
    # 'scale_pos_weight':3.4,
    'random_state': 42
    
}


evals = [(dtrain, 'train'), (deval, 'eval')]
xgb_model = xgb.train(
    params,
    dtrain,
    num_boost_round= 800,
    early_stopping_rounds= 30,
    evals=evals,
    verbose_eval=10
)

X_train_pred = X_train_new.copy()
X_eval_pred = X_eval.copy()
X_test_pred = X_test.copy()
X_oot_pred = X_oot.copy()


X_train_pred['y_actual'] = y_train_new
X_train_pred['y_pred'] = xgb_model.predict(dtrain)
X_train_pred['LID'] = model_df1.loc[X_train_new.index,'latest_feature_id'].values

X_eval_pred['y_actual'] = y_eval
X_eval_pred['y_pred'] = xgb_model.predict(deval)
X_eval_pred['LID'] = model_df1.loc[X_eval.index,'latest_feature_id'].values

X_test_pred['y_actual'] = y_test
X_test_pred['y_pred'] = xgb_model.predict(dtest)
X_test_pred['LID'] = model_df1.loc[X_test.index,'latest_feature_id'].values

X_oot_pred['y_actual'] = y_oot
X_oot_pred['y_pred'] = xgb_model.predict(doot)
X_oot_pred['LID'] = oot_df1.loc[X_oot.index,'latest_feature_id'].values

[0]	train-auc:0.61504	eval-auc:0.61546
[10]	train-auc:0.64113	eval-auc:0.63656
[20]	train-auc:0.64360	eval-auc:0.63935
[30]	train-auc:0.64380	eval-auc:0.63886
[40]	train-auc:0.64838	eval-auc:0.64364
[50]	train-auc:0.65040	eval-auc:0.64429
[60]	train-auc:0.65014	eval-auc:0.64572
[70]	train-auc:0.65021	eval-auc:0.64581
[80]	train-auc:0.65161	eval-auc:0.64757
[90]	train-auc:0.65310	eval-auc:0.64936
[100]	train-auc:0.65423	eval-auc:0.65083
[110]	train-auc:0.65558	eval-auc:0.65151
[120]	train-auc:0.65585	eval-auc:0.65209
[130]	train-auc:0.65688	eval-auc:0.65249
[140]	train-auc:0.65800	eval-auc:0.65297
[150]	train-auc:0.65856	eval-auc:0.65325
[160]	train-auc:0.65883	eval-auc:0.65361
[170]	train-auc:0.65975	eval-auc:0.65467
[180]	train-auc:0.66038	eval-auc:0.65511
[190]	train-auc:0.66105	eval-auc:0.65521
[200]	train-auc:0.66173	eval-auc:0.65554
[210]	train-auc:0.66232	eval-auc:0.65594
[220]	train-auc:0.66302	eval-auc:0.65675
[230]	train-auc:0.66348	eval-auc:0.65725
[240]	train-auc:0.66422	eva

In [34]:
# 6. **Run Decile Analysis**
train_summary, train_auc, train_gini, train_ks = decile_analysis(X_train_pred, 'y_pred', 'y_actual')
eval_summary, eval_auc, eval_gini, eval_ks = decile_analysis(X_eval_pred, 'y_pred', 'y_actual')

test_summary, test_auc, test_gini, test_ks = decile_analysis(X_test_pred, 'y_pred', 'y_actual')
oot_summary, oot_auc, oot_gini, oot_ks = decile_analysis(X_oot_pred, 'y_pred', 'y_actual')

# 7. **Print Results**
print(f"Train AUC: {train_auc:.2f}, Gini: {train_gini:.2f}, KS: {train_ks:.2f}")
print(f"Eval AUC: {eval_auc:.2f}, Gini: {eval_gini:.2f}, KS: {eval_ks:.2f}")
print(f"Test AUC: {test_auc:.2f}, Gini: {test_gini:.2f}, KS: {test_ks:.2f}")
print(f"OOT AUC: {oot_auc:.2f}, Gini: {oot_gini:.2f}, KS: {oot_ks:.2f}")


Train AUC: 0.68, Gini: 0.36, KS: 0.27
Eval AUC: 0.67, Gini: 0.34, KS: 0.24
Test AUC: 0.66, Gini: 0.33, KS: 0.23
OOT AUC: 0.68, Gini: 0.35, KS: 0.26


In [35]:
print(f"Train AUC: {train_auc:.2f}, Gini: {train_gini:.4f}, KS: {train_ks:.4f}")
print(f"Test AUC: {test_auc:.2f}, Gini: {test_gini:.4f}, KS: {test_ks:.4f}")
print(f"OOT AUC: {oot_auc:.2f}, Gini: {oot_gini:.4f}, KS: {oot_ks:.4f}")

Train AUC: 0.68, Gini: 0.3628, KS: 0.2658
Test AUC: 0.66, Gini: 0.3262, KS: 0.2349
OOT AUC: 0.68, Gini: 0.3518, KS: 0.2630


In [36]:
feature_importance = xgb_model.get_score(importance_type='weight')
feature_importance_df = pd.DataFrame(
    list(feature_importance.items()),
    columns =['Feature','Importance']
).sort_values(by='Importance',ascending=False)
print(feature_importance_df)

                                              Feature  Importance
14                          Num_Enq_non_BANK_last_12M       240.0
9                   num_PL_LE_1_Lac_opened_in_last_3M       220.0
5                highest_sanction_amount_credit_cards       211.0
0                           Num_Enq_all_loans_last_6M       180.0
12                       percentage_utilization_gt_75       156.0
7             Utilization_CREDIT CARD_opened_last_36M       144.0
6                          Age_of_oldest_open_account       138.0
2                       num_CONSUMER LOAN_enq_last_9M       136.0
4   Total_outstanding_Balance_CREDIT CARD_opened_l...       130.0
8                               total_credit_all_open       129.0
10                         percentage_ever_delinquent       117.0
15                     Num_of_installment_loan_opened       115.0
13                           No of enq in past 90days       111.0
11   months_since_most_recent_delinquency_installment        90.0
3         

In [37]:
feature_importance_df

Feature  Importance
14                          Num_Enq_non_BANK_last_12M       240.0
9                   num_PL_LE_1_Lac_opened_in_last_3M       220.0
5                highest_sanction_amount_credit_cards       211.0
0                           Num_Enq_all_loans_last_6M       180.0
12                       percentage_utilization_gt_75       156.0
7             Utilization_CREDIT CARD_opened_last_36M       144.0
6                          Age_of_oldest_open_account       138.0
2                       num_CONSUMER LOAN_enq_last_9M       136.0
4   Total_outstanding_Balance_CREDIT CARD_opened_l...       130.0
8                               total_credit_all_open       129.0
10                         percentage_ever_delinquent       117.0
15                     Num_of_installment_loan_opened       115.0
13                           No of enq in past 90days       111.0
11   months_since_most_recent_delinquency_installment        90.0
3                         num_CREDIT CARD_enq_last_3M        55.0
1                      num_PERSONAL LOAN_enq_last_36M        11.0

In [38]:
train_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.268382         0.251041        0.582407        0.180647   4896   
9          0.166088         0.157454        0.180642        0.140030   4895   
8          0.137692         0.128505        0.140022        0.118322   4895   
7          0.108478         0.110254        0.118320        0.102734   4895   
6          0.096221         0.096265        0.102734        0.090553   4895   
5          0.085691         0.085037        0.090549        0.079755   4878   
4          0.070351         0.075597        0.079754        0.071660   4904   
3          0.062513         0.068375        0.071660        0.064945   4895   
2          0.047726         0.059912        0.064943        0.054072   4903   
1          0.035948         0.046991        0.054070        0.034938   4896   

        label_good  label_bad  
Decile                         
10            3582       1314  
9             4082        813  
8             4221        674  
7             4364        531  
6             4424        471  
5             4460        418  
4             4559        345  
3             4589        306  
2             4669        234  
1             4720        176

In [39]:
test_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.262500         0.249220        0.575852        0.180467   4080   
9          0.154204         0.157878        0.180446        0.140289   4079   
8          0.135572         0.128586        0.140288        0.118433   4079   
7          0.116667         0.109968        0.118429        0.102391   4080   
6          0.100515         0.096089        0.102391        0.090608   4079   
5          0.095857         0.085150        0.090605        0.079872   4079   
4          0.077415         0.075679        0.079870        0.071726   4069   
3          0.066259         0.068333        0.071725        0.064924   4090   
2          0.056877         0.059914        0.064923        0.054213   4079   
1          0.040441         0.047180        0.054202        0.035019   4080   

        label_good  label_bad  
Decile                         
10            3009       1071  
9             3450        629  
8             3526        553  
7             3604        476  
6             3669        410  
5             3688        391  
4             3754        315  
3             3819        271  
2             3847        232  
1             3915        165

In [40]:
oot_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.251873         0.262616        0.562546        0.186521   1735   
9          0.155709         0.161523        0.186386        0.142385   1734   
8          0.148789         0.129612        0.142380        0.118534   1734   
7          0.105536         0.110025        0.118521        0.102517   1734   
6          0.096309         0.096369        0.102511        0.090738   1734   
5          0.091329         0.085075        0.090733        0.079758   1730   
4          0.064442         0.075825        0.079754        0.071930   1738   
3          0.056484         0.068637        0.071928        0.065427   1735   
2          0.052480         0.060538        0.065414        0.054484   1734   
1          0.036888         0.047373        0.054481        0.034889   1735   

        label_good  label_bad  
Decile                         
10            1298        437  
9             1464        270  
8             1476        258  
7             1551        183  
6             1567        167  
5             1572        158  
4             1626        112  
3             1637         98  
2             1643         91  
1             1671         64

In [41]:
#Save the model
xgb_model.save_model('LeoPayu_Bureau_60_in_15mob.json')
print('Model Saved')

Model Saved


In [43]:
#Save the Pred
Train = X_train_pred[['LID','y_actual','y_pred']]
Eval = X_eval_pred[['LID','y_actual','y_pred']]
Test = X_test_pred[['LID','y_actual','y_pred']]
Oot = X_oot_pred[['LID','y_actual','y_pred']]


In [44]:
Train.to_csv('LeoPayu_Bureau_Train_Pred.csv',index=False)
Eval.to_csv('LeoPayu_Bureau_Eval_Pred.csv',index=False)
Test.to_csv('LeoPayu_Bureau_Test_Pred.csv',index=False)
Oot.to_csv('LeoPayu_Bureau_Oot_Pred.csv',index=False)
